In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [2]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Maun/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Maun/metadata_107.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,1_1.wav,1,107,AL-Maun
1,1_2.wav,1,107,AL-Maun
2,1_3.wav,1,107,AL-Maun
3,1_4.wav,1,107,AL-Maun
4,1_5.wav,1,107,AL-Maun
5,1_6.wav,1,107,AL-Maun
6,1_7.wav,1,107,AL-Maun
7,1.wav,3,107,AL-Maun
8,2.wav,3,107,AL-Maun
9,3.wav,3,107,AL-Maun


Feature Extraction

In [3]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [4]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [5]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00109242  0.00021382 -0.00252546 ... -0.04498462 -0.023406
  0.00867686] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.03584673 0.0407787  0.05798257 ... 0.04174525 0.03500508 0.02826281] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.02566469  0.0255555   0.00288212 ... -0.12978713 -0.19195503
 -0.29417735] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.00545041 -0.03084549 -0

In [6]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (7, 704, 40)
Shape of labels for fold 1: (7,)


In [7]:
mfcc_data_by_fold[3]["data"].shape

(7, 704, 40)

In [8]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [9]:
mfcc_data_by_fold[1]["data"].shape

(7, 704, 40, 1)

After Training Work, Deep Features 

In [10]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 81ms/step


In [11]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [12]:
intermediate_layer_outputs.shape

(3, 7, 1024)

In [13]:
intermediate_layer_outputs

array([[[ 0.90965   ,  0.9999843 ,  0.91977894, ...,  0.7957634 ,
         -0.9944824 , -0.9985868 ],
        [-0.8904938 ,  0.9999464 ,  0.9970605 , ...,  0.99860084,
         -0.99845487, -0.9998161 ],
        [-0.54282576,  0.87520814,  0.9941514 , ...,  0.99883264,
         -0.9979167 , -0.99939066],
        ...,
        [-0.71480477,  0.9999946 ,  0.9998074 , ...,  0.99545676,
         -0.99998766, -0.9998534 ],
        [-0.9278641 ,  0.9984845 ,  0.9573803 , ...,  0.9992651 ,
         -0.9999968 , -0.9999332 ],
        [-0.59586394,  0.99181277,  0.99835855, ...,  0.97901946,
         -0.9959859 , -0.9999657 ]],

       [[-0.44979218, -0.95935947,  0.99999535, ...,  0.9913798 ,
         -0.9999998 , -0.9999809 ],
        [-0.5699204 ,  0.99999845,  0.9993421 , ...,  0.9996192 ,
         -0.99999785, -0.99995154],
        [-0.9992826 , -0.9753085 ,  0.99976414, ...,  0.9994041 ,
         -1.        , -0.9999849 ],
        ...,
        [-0.9999465 , -0.968181  ,  1.        , ...,  

In [14]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [15]:
al_maun_mean = mean_across_folds

In [16]:
al_maun_mean.shape

(7, 1024)

In [17]:
al_maun_mean

array([[-0.1378548 , -0.27396324,  0.9722407 , ...,  0.9278318 ,
        -0.9979868 , -0.99937177],
       [-0.4505113 ,  0.33503246,  0.99346644, ...,  0.9951356 ,
        -0.99946135, -0.99569017],
       [-0.84291124, -0.27151605,  0.99794054, ...,  0.999404  ,
        -0.9992872 , -0.9997072 ],
       ...,
       [-0.83209467,  0.34388885,  0.9947929 , ...,  0.99637365,
        -0.99986607, -0.9998787 ],
       [-0.90350896,  0.33264944,  0.9828374 , ...,  0.99964494,
        -0.9999363 , -0.9999375 ],
       [-0.77766275, -0.32544485,  0.9912564 , ...,  0.9883339 ,
        -0.99841195, -0.9977746 ]], dtype=float32)

In [19]:
# Specify the file path where you want to save the .npy file
file_path = 'al_maun_mean.npy'

# Save the array as a .npy file
np.save(file_path, al_maun_mean)

Testing of Single Audio

In [20]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/maun_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-1.5829317e-04  6.3124535e-05  4.4476031e-04 ...  9.8179437e-02
  4.8970614e-02 -5.3284939e-02] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.132186    0.16677025  0.19307207 ... -0.27347195 -0.19301663
 -0.17537782] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.06666883  0.07902554  0.06253501 ...  0.26959693  0.12902094
 -0.06139604] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.15267949 -0.14

In [21]:
mfcc_extracted_features_single

array([[[[-5.2517548e+02],
         [ 3.6440083e+01],
         [ 2.1608559e+01],
         ...,
         [-1.0742366e-01],
         [ 2.9936929e+00],
         [ 1.0349838e+01]],

        [[-5.0890161e+02],
         [ 4.5533978e+01],
         [ 2.1682173e+01],
         ...,
         [-2.6655667e+00],
         [-1.5828061e+00],
         [ 6.8974905e+00]],

        [[-5.1226141e+02],
         [ 4.5336964e+01],
         [ 2.2660995e+01],
         ...,
         [-4.9264660e-01],
         [-8.5377872e-02],
         [ 2.1273274e+00]],

        ...,

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         [ 0.0000000e+00]],

        [[-1.1313708e+03],
         [ 0.0000000e+00],
         [ 0.0000000e+00],
         ...,
 

In [22]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 138ms/step


In [23]:
intermediate_output

array([[-0.78266513,  0.9994637 ,  0.99113184, ...,  0.999818  ,
        -0.99993855, -0.99987894]], dtype=float32)

In [24]:
intermediate_output.shape

(1, 1024)

In [26]:
intermediate_layer_outputs

array([[[ 0.90965   ,  0.9999843 ,  0.91977894, ...,  0.7957634 ,
         -0.9944824 , -0.9985868 ],
        [-0.8904938 ,  0.9999464 ,  0.9970605 , ...,  0.99860084,
         -0.99845487, -0.9998161 ],
        [-0.54282576,  0.87520814,  0.9941514 , ...,  0.99883264,
         -0.9979167 , -0.99939066],
        ...,
        [-0.71480477,  0.9999946 ,  0.9998074 , ...,  0.99545676,
         -0.99998766, -0.9998534 ],
        [-0.9278641 ,  0.9984845 ,  0.9573803 , ...,  0.9992651 ,
         -0.9999968 , -0.9999332 ],
        [-0.59586394,  0.99181277,  0.99835855, ...,  0.97901946,
         -0.9959859 , -0.9999657 ]],

       [[-0.44979218, -0.95935947,  0.99999535, ...,  0.9913798 ,
         -0.9999998 , -0.9999809 ],
        [-0.5699204 ,  0.99999845,  0.9993421 , ...,  0.9996192 ,
         -0.99999785, -0.99995154],
        [-0.9992826 , -0.9753085 ,  0.99976414, ...,  0.9994041 ,
         -1.        , -0.9999849 ],
        ...,
        [-0.9999465 , -0.968181  ,  1.        , ...,  

Euclidian Distance with Mean

In [27]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(al_maun_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = al_maun_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in al_maun_mean: {closest_index+1}")
print(f"Closest value in al_maun_mean: {closest_value}")

Index of closest value in al_fil_mean: 6
Closest value in al_fil_mean: [-0.90350896  0.33264944  0.9828374  ...  0.99964494 -0.9999363
 -0.9999375 ]


Euclidian Distance Without Mean

In [26]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")